In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [62]:
!ls /content/drive/My\ Drive/Tesis/Data/New_FCBF/graficos/

1esp_15hz_1hz_1  2021_GRAF5  archive
1esp_15hz_1hz_2  2021_GRAF6  dep_writing_nf100_bwf1_fi0.5_bdfb15_esp1.csv
2021_GRAF1	 2021_GRAF7  oth_ith_1
2021_GRAF2	 2021_GRAF8  oth_ith_2
2021_GRAF3	 6.ipynb     oth_ith_3
2021_GRAF4	 7.ipynb


In [74]:
for name in dir():
       if not name.startswith('_'):
           del globals()[name]

# Reset Variables

In [75]:
# Importar librerías
import pandas as pd
import numpy as np
import math
import statistics as sta
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB 
from random import seed
from random import randint
import random

from sklearn.metrics import roc_auc_score
from sklearn import metrics       
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve           

from sklearn import metrics 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve                         # thresholds = all unique prediction probabilities in descending order

import time

In [76]:
def agument_Data(x, Eps, proportion):
  N=int(proportion*len(x))
  df_to_be_agumented=x
  rn=rng.normal(0, Eps, size=(1, x.shape[1]))
  #print('rn=',rn)

  for i in range(N):                            # create new N data. Where N are the required proportion.
    rd_row=random.randint(0,len(x)-1)           # randomly select one row to include noise.
    #print('i',i,'rd_row',rd_row)
    b=x.values[rd_row,0:x.shape[1]]+rn          # add Gaussian Noise to the selected observation.
    b=pd.DataFrame(data=b)                      # return format to Panda Dataframe
    b.columns = x.columns                       # re-name the new dataframe with the original names of the columns
    b.values[0,b.shape[1]-1]=int(x.values[rd_row,b.shape[1]-1]) # convert to integer the last value (this is the value used to classify)
    df_to_be_agumented = pd.concat([df_to_be_agumented,b])      # concat the augmente observation at the end of the array.
    df_to_be_agumented.reset_index(inplace=True, drop=True)     # reset the index
  return(df_to_be_agumented)


In [77]:

path = "/content/drive/My Drive/Tesis/Data/New_FCBF/graficos/"
file= path + "dep_writing_nf100_bwf1_fi0.5_bdfb15_esp1.csv"
data = pd.read_csv(file, header = None)
data_name = []
for i in range(len(data.axes[1])):

  if i == len(data.axes[1])-1:
    data_name = data_name + ['Emotions']
  else:
    data_name = data_name + [str(i)]

data_n = data_name
data.columns = data_name
def make_dataframes_classes_same_length(x, Eps, prop):
  e0=x[(x['Emotions']==0)]
  e1=x[(x['Emotions']==1)]
  l_e0=len(e0)
  l_e1=len(e1)
  #print('e0',len(e0))
  #print('e1',len(e1))
  if l_e0>l_e1:
      prop=l_e0/l_e1-1
      e1=agument_Data(e1, Eps, prop)
  else:
      prop=e1/l_e0-1
      e0=agument_Data(e0,prop, Eps)
  df_to_be_agumented = pd.concat([e0,e1])                   # concat the augmente observation at the end of the array.
  df_to_be_agumented.reset_index(inplace=True, drop=True)   # reset the index
  return(df_to_be_agumented)


In [78]:
e0=data[(data['Emotions']==0)]

In [79]:
####################################################################################
# suffle_df: retruns the dataframes with segments i and j of length k shuffled.
#
# Input:
#   X is the input matrix to be shuffled.
#   i is the starting rows of the first segment of rows to shuffle
#   j is the starting rows of the second segment of rows to shuffle
#   k is the number of rows to shuffle
#  Output:
#   the dataframes with segments i and j of length k shuffled.
####################################################################################
def suffle_df(X, i, j, k):
  #a=X.iloc[0:8].copy()
  a=X
  #print(a)
  b,c=a.iloc[0:i+k].copy(),a.iloc[j:j+k].copy()
  b.reset_index(inplace=True, drop=True)
  c.reset_index(inplace=True, drop=True)
  b.index=b.index + j  # modify the values of the index such that 'b' variable can be replaced.
  #print(b)
  #print(c)
  a.iloc[0:i+k], a.iloc[j:j+k]=c,b
  return(a)


In [80]:
####################################################################################
####################################################################################
####################################################################################
def train_test_sets(x, i, k, Eps, pr):
  X_test=x.values[i:k,col]
  y_test=x.values[i:k,nf]
  tmp=x.loc[k:len(x)]
  #print('Input tmp.shape[0]',tmp.shape[0])
  tmp = make_dataframes_classes_same_length(tmp, Eps, pr)
  #tmp = agument_Data(tmp, Eps, pr)
  #print('Output tmp.shape[0]', tmp.shape[0])
  X_train=tmp.values[k:len(tmp),col]
  y_train=tmp.values[k:len(tmp),nf]
  #print('X_train.shape[0]', X_train.shape[0])
  #print('y_train.shape[0]', y_train.shape[0])

  return X_train, X_test, y_train, y_test


In [81]:
def normalize(dataset):
    dataNorm=(dataset-dataset.mean())/(dataset.std())
    dataNorm["Emotions"]=dataset["Emotions"]
    return dataNorm
data=normalize(data)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,Emotions
0,-1.712001,-1.182212,-0.636509,-1.130894,-0.347844,0.620320,-0.258261,-0.743037,-0.537440,-0.785424,-0.984541,-1.106182,-1.185749,-0.819819,1.836050,1.283038,2.683674,2.683674,1.283038,1.836050,-1.443963,-1.455934,-1.463109,-1.470021,-1.489114,-1.495351,-1.516349,-1.475833,0.580387,0.562556,0.823518,0.823518,0.562556,0.580387,-2.359926,-2.399073,-2.334314,-2.422314,-2.434719,-2.607525,...,0.491823,0.773210,0.266821,0.890372,0.580647,-0.209448,-1.366393,-0.045370,-0.045370,-1.366393,-0.209448,0.281086,0.261975,0.246202,0.235589,0.239499,0.245870,0.263407,0.253865,0.070246,0.155949,0.136786,0.136786,0.155949,0.070246,0.571178,0.395700,0.741260,0.691243,0.767086,0.658973,0.458790,0.645813,-0.371630,-0.717350,-1.076166,-1.076166,-0.717350,-0.371630,0
1,-1.685251,-0.094765,0.686864,0.168890,-0.503110,1.990057,0.544777,0.954085,0.510356,0.606745,0.645134,0.645337,0.653319,0.667041,0.376778,0.308851,-1.483147,-1.483147,0.308851,0.376778,-0.131370,-0.143375,-0.155675,-0.172341,-0.189465,-0.209096,-0.230479,-0.175350,1.230055,1.232104,1.198511,1.198511,1.232104,1.230055,0.828422,0.799023,0.796270,0.799320,0.937160,1.003150,...,1.804095,1.503629,2.178756,2.175462,1.679254,-1.204785,-2.158262,-1.138107,-1.138107,-2.158262,-1.204785,1.835983,1.856214,1.876337,1.898381,1.920955,1.934769,1.953836,1.899105,-0.349334,-0.333419,-0.396546,-0.396546,-0.333419,-0.349334,1.771202,1.843347,2.165370,1.735817,1.995509,1.999020,1.682617,1.980145,-0.820760,-0.097554,-0.620167,-0.620167,-0.097554,-0.820760,0
2,-1.658501,-0.063935,-0.326338,-0.163837,-0.037312,1.280243,0.125951,-0.237787,-0.254928,-0.024466,-0.092434,-0.142681,-0.218711,-0.121294,-0.094191,1.064255,1.095848,1.095848,1.064255,-0.094191,-0.257115,-0.254046,-0.251124,-0.243568,-0.243059,-0.231465,-0.232719,-0.245303,-0.404660,-0.401519,-0.297296,-0.297296,-0.401519,-0.404660,-0.453985,-0.481240,-0.394658,-0.426302,-0.500463,-0.525378,...,0.537254,0.263743,0.811868,0.887073,0.665155,-0.315806,-1.111686,-0.549576,-0.549576,-1.111686,-0.315806,1.031695,1.053697,1.079825,1.109784,1.145251,1.185579,1.230159,1.119438,-1.055173,-1.047982,-1.038433,-1.038433,-1.047982,-1.055173,1.805139,1.586275,0.869863,0.735168,0.419923,0.817280,1.439300,1.147756,1.635017,-0.658546,-0.981280,-0.981280,-0.658546,1.635017,0
3,-1.631751,-1.278684,-0.092423,-1.013534,-0.192578,1.140187,-0.625166,-1.089587,-0.791343,-1.378061,-1.406009,-1.212134,-1.399516,-1.154114,1.726848,0.170173,3.141220,3.141220,0.170173,1.726848,-1.060113,-1.068585,-1.076872,-1.087172,-1.098326,-1.110593,-1.122075,-1.087914,0.567731,0.577602,0.553615,0.553615,0.577602,0.567731,-3.834190,-3.863693,-3.648355,-3.655440,-3.880183,-3.965917,...,0.495317,1.036161,0.591457,0.915117,0.446599,-0.873905,-1.132234,-1.658261,-1.658261,-1.132234,-0.873905,0.756143,0.769397,0.785658,0.804727,0.820133,0.842277,0.866010,0.807051,-0.527243,-0.568693,-0.532264,-0.532264,-0.568693,-0.527243,-0.008471,-0.460249,0.125319,0.376910,-0.280025,0.067409,-0.643378,-0.115468,-0.420596,-0.024501,0.177214,0.177214,-0.024501,-0.420596,1
4,-1.605001,-0.346418,-0.081805,-0.294551,-0.347844,0.457422,-0.583630,-0.464502,-0.684060,-0.929200,-0.542000,-0.665819,-0.761610,-0.675125,0.388369,0.730051,-0.018130,-0.018130,0.730051,0.388369,-1.307189,-1.303207,-1.298847,-1.296403,-1.290371,-1.285077,-1.278889,-1.294593,-0.592389,-0.605393,-0.619050,-0.619050,-0.605393,-0.592389,-0.262129,-0.209824,-0.223808,-0.313744,-0.356701,-0.212111,...,0.047992,0.639909,1.367166,1.312687,1.026501,-0.580915,-0.583303,0.183538,0.183538,-0.583303,-0.580915,1.368082,1.382981,1.400240,1.417574,1.441978,1.461693,1.482475,1.424016,-0.462143,-0.422158,-0.435976,-0.435976,-0.422158,-0.462143,1.544500,1.574785,1.234013,1.211471,0.69

In [82]:
#PCA
def PComp(dataset,n):
  fDum = dataset.columns
  features = fDum[0:-1] 
  x = dataset.loc[:,features].values
  y = dataset.loc[:,["Emotions"]].values
  pca = PCA(n_components=n)
  prinComp = pca.fit_transform(x)
  names = []
  for i in range(n):
      names = names + ["P Component" + str(i+1)]
  prinDf = pd.DataFrame(data = prinComp, columns = names)
  finalDf = pd.concat([prinDf,dataset[["Emotions"]]], axis = 1)
  return finalDf,pca
data,pca = PComp(data, 129)
data

,P Component1,P Component2,P Component3,P Component4,P Component5,P Component6,P Component7,P Component8,P Component9,P Component10,P Component11,P Component12,P Component13,P Component14,P Component15,P Component16,P Component17,P Component18,P Component19,P Component20,P Component21,P Component22,P Component23,P Component24,P Component25,P Component26,P Component27,P Component28,P Component29,P Component30,P Component31,P Component32,P Component33,P Component34,P Component35,P Component36,P Component37,P Component38,P Component39,P Component40,...,P Component91,P Component92,P Component93,P Component94,P Component95,P Component96,P Component97,P Component98,P Component99,P Component100,P Component101,P Component102,P Component103,P Component104,P Component105,P Component106,P Component107,P Component108,P Component109,P Component110,P Component111,P Component112,P Component113,P Component114,P Component115,P Component116,P Component117,P Component118,P Component119,P Component120,P Component121,P Component122,P Component123,P Component124,P Component125,P Component126,P Component127,P Component128,P Component129,Emotions
0,-6.436870,-7.888150,1.596000,6.700599,5.505204,-2.026932,4.192070,2.994766,-1.474337,-1.397808,4.313097,-2.310562,-5.674048,-3.167921,-1.654487,1.355795,-1.835461,-1.526483,-2.710111,-0.832512,1.991376,-0.278817,-1.155104,1.286399,-2.080521,-2.423147,0.468834,-0.088987,0.272424,-2.581416,-1.126467,-1.251897,-2.658872,1.103682,1.303571,-2.590372,-1.220052,-0.340860,-0.986219,0.005834,...,0.168924,-0.146669,0.033538,-0.070104,-0.062116,0.015014,-0.083900,-0.056623,-0.102339,-0.101133,-0.002060,-0.022745,0.010904,0.056328,-0.005169,0.008546,0.037217,0.006098,-0.043129,0.017610,-0.009577,0.005849,0.019290,0.009544,0.008552,0.013139,-0.019123,0.011317,0.006101,0.001383,0.008633,0.008423,-0.001969,0.006291,0.001595,-0.006297,-0.002519,-0.000870,6.900929e-16,0
1,6.286582,-8.140600,-8.319126,1.365120,0.173661,-5.152554,1.052757,2.714616,-3.103328,-1.515486,-1.646286,-1.958254,0.864676,0.842992,0.893062,0.375097,-1.051727,-1.354981,-0.335367,1.074569,0.878246,1.649285,-0.914153,-1.475251,-0.703578,-0.289670,-0.992626,0.299235,0.551252,-0.652593,2.813453,0.504844,0.424127,0.190495,0.439304,0.016341,1.183266,-0.021266,0.900943,-0.034870,...,0.216203,-0.076196,-0.082191,0.043373,0.016328,-0.055731,0.015843,0.150333,-0.047826,-0.145275,-0.094729,0.027584,0.044467,-0.024278,0.070457,-0.030132,0.008012,0.058624,-0.025282,0.057317,0.029282,-0.030883,-0.026366,-0.052229,-0.011218,0.033199,0.021938,0.036805,-0.022303,0.021127,-0.007424,-0.016404,0.003734,-0.003666,-0.005801,0.012087,-0.009282,-0.002384,6.900929e-16,0
2,2.870617,-8.083723,-1.100933,0.722050,-3.880194,-3.012298,-1.524296,-0.609808,-2.364628,-2.280802,-1.899642,-1.588067,-2.034633,1.472604,1.991385,-4.167869,-1.950193,-2.202692,0.499856,-0.701452,-1.053443,1.390326,0.316521,1.958839,1.770756,-2.074406,0.463867,-0.044609,2.624600,-0.925831,-0.468722,1.186029,-0.798708,-1.562316,-0.116540,0.802107,-0.536657,-0.771861,0.738268,-1.359694,...,0.005956,-0.039882,-0.028702,-0.034103,0.135388,-0.061281,0.096806,-0.040153,0.077605,0.077200,0.050708,-0.010680,0.010448,0.012491,-0.036620,-0.088660,-0.037959,0.000304,0.046318,0.025158,0.003982,0.016153,0.000452,-0.010804,-0.041244,-0.000447,-0.007341,0.001641,-0.008116,-0.011685,0.001480,0.042392,0.002380,-0.012286,0.017935,-0.000245,-0.001380,0.002470,6.900929e-16,0
3,-2.986365,-10.307549,8.595439,2.757807,3.470946,-3.662179,6.615813,-4.912160,6.639420,-7.967940,-1.579977,-2.013770,-4.343596,-6.210954,2.040881,1.414542,-1.423363,1.075867,0.584011,-1.792508,-1.221605,-1.855732,-1.625680,-0.584785,-0.766384,-0.326913,0.862666,-1.638859,1.057847,0.163060,0.261273,-0.954876,-1.294714,-0.636812,-2.189910,1.793994,0.689110,0.179125,-1.182561,0.486119,...,-0.157584,-0.083316,-0.074059,-0.030497,0.039675,-0.018478,0.034925,-0.025233,0.029988,0.041912,0.053038,-0.027714,0.023547,-0.038227,-0.047488,-0.014100,-0.047405,-0.018300,-

In [83]:
lst = []
lst = [index for index,value in enumerate(pca.explained_variance_) if value > 1]
data,pca = PComp(data, len(lst))
data

,P Component1,P Component2,P Component3,P Component4,P Component5,P Component6,P Component7,P Component8,P Component9,P Component10,P Component11,P Component12,P Component13,P Component14,P Component15,P Component16,P Component17,P Component18,P Component19,P Component20,P Component21,P Component22,P Component23,P Component24,P Component25,P Component26,P Component27,P Component28,P Component29,P Component30,P Component31,P Component32,P Component33,P Component34,P Component35,P Component36,P Component37,P Component38,Emotions
0,-6.436870,-7.888150,1.596000,6.700599,5.505204,-2.026932,4.192070,2.994766,-1.474337,-1.397808,4.313097,-2.310562,-5.674048,-3.167921,-1.654487,1.355795,-1.835461,-1.526483,-2.710111,-0.832512,1.991376,-0.278817,-1.155104,1.286399,-2.080521,-2.423147,0.468834,-0.088987,0.272424,-2.581416,-1.126467,-1.251897,-2.658872,1.103682,1.303571,-2.590372,-1.220052,-0.340860,0
1,6.286582,-8.140600,-8.319126,1.365120,0.173661,-5.152554,1.052757,2.714616,-3.103328,-1.515486,-1.646286,-1.958254,0.864676,0.842992,0.893062,0.375097,-1.051727,-1.354981,-0.335367,1.074569,0.878246,1.649285,-0.914153,-1.475251,-0.703578,-0.289670,-0.992626,0.299235,0.551252,-0.652593,2.813453,0.504844,0.424127,0.190495,0.439304,0.016341,1.183266,-0.021266,0
2,2.870617,-8.083723,-1.100933,0.722050,-3.880194,-3.012298,-1.524296,-0.609808,-2.364628,-2.280802,-1.899642,-1.588067,-2.034633,1.472604,1.991385,-4.167869,-1.950193,-2.202692,0.499856,-0.701452,-1.053443,1.390326,0.316521,1.958839,1.770756,-2.074406,0.463867,-0.044609,2.624600,-0.925831,-0.468722,1.186029,-0.798708,-1.562316,-0.116540,0.802107,-0.536657,-0.771861,0
3,-2.986365,-10.307549,8.595439,2.757807,3.470946,-3.662179,6.615813,-4.912160,6.639420,-7.967940,-1.579977,-2.013770,-4.343596,-6.210954,2.040881,1.414542,-1.423363,1.075867,0.584011,-1.792508,-1.221605,-1.855732,-1.625680,-0.584785,-0.766384,-0.326913,0.862666,-1.638859,1.057847,0.163060,0.261273,-0.954876,-1.294714,-0.636812,-2.189910,1.793994,0.689110,0.179125,1
4,3.087332,-8.288729,3.534635,5.250708,-1.612861,-0.558505,-4.306481,2.190690,-1.425761,-2.868453,-0.373827,-1.635162,1.105724,1.258888,1.810402,-1.055950,-0.558489,-0.904830,0.271826,0.800974,-0.024542,0.243722,-1.535327,-0.072164,1.048577,-0.235112,-0.021396,0.089824,1.006948,0.288910,-1.262740,0.724823,0.880536,-1.114212,0.418485,-0.436401,0.532697,-0.679198,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.385799,-3.399548,2.936621,-6.632693,-1.043072,-2.938318,-2.377721,-0.638137,1.899067,-2.489178,1.684214,-3.670067,-2.216672,0.310498,-1.326204,3.788225,5.682454,-0.841522,-0.520003,1.204573,-0.940636,1.184352,1.782381,-1.999556,-3.283430,-1.034125,0.624926,-0.505433,0.354674,-1.555287,0.797014,1.142159,-1.657199,0.616065,-1.456880,-0.087523,0.445154,1.047219,1
125,0.288800,-1.387912,2.542312,-3.828118,0.084755,-4.595685,3.395649,-0.187798,-0.177384,-2.163836,-1.750235,-1.702216,1.437133,-0.368183,-0.882548,2.010931,-1.702859,3.115427,0.984406,-0.203934,-0.711458,-0.677458,0.481624,0.267720,-1.974573,0.058574,2.641280,1.461836,0.831469,-1.337339,1.412248,-0.919942,-0.824144,-0.461568,-1.030304,-0.277698,-0.311421,-0.431497,1
126,7.683617,1.504807,5.872466,-2.682965,-1.734609,1.420836,-3.458604,3.209393,1.883839,-1.095638,0.127240,-3.007893,0.881331,3.260222,-0.798180,-0.372720,-3.446795,1.800425,-0.450597,0.504227,0.690976,-2.048871,-0.889463,-2.503392,1.943200,-0.195646,4.010052,0.561201,-0.597714,1.685029,-0.218469,-0.391721,-1.566977,0.324470,-0.284255,1.719142,0.005165,0.239765,1
127,5.749276,1.141870,-8.445863,-3.366787,3.571734,3.522023,-3.970517,-0.475238,0.201957,1.844622,0.583533,6.167772,-3.245969,-1.125611,3.674697,-1.432835,4.620549,-1.847361,-0.642909,0.698789,-0.494117,-4.187344,0.413688,1.061060,0.568097,0.106860,-2.027951,-1.201622,-0.062613,1.122032,-2.387876,-2.529395,-0.237629,0.777472,-0.953248,2.223266,-0.232709,1.038

# Training and Testing Function

In [ ]:
####################################################################################
####################################################################################
####################################################################################
start_time=time.time()

LR_max_iter=500

#Eps=0.1; prop=0.2; **w= 23 j= 1 average accuracy= 0.6640625
Eps=0.2 #por la normalización
prop=0.2

CV=5
k=1
acc_max=0
for w in range(200,400):
   for j in range(200,400):
     path = "/content/drive/My Drive/Tesis/Data/New_FCBF/graficos/"
     file= path + "dep_writing_nf100_bwf1_fi0.5_bdfb15_esp1.csv"

     data = pd.read_csv(file, header = None)
     data_name = []
     for i in range(len(data.axes[1])):
      if i== len(data.axes[1])-1:
        data_name = data_name + ['Emotions']
      else:
        data_name = data_name + [str(i)]

     data_n = data_name
     data.columns = data_name
     data=normalize(data) 
     data,_ = PComp(data, len(lst))
     nf = data.shape[1]-1 # número de features (componentes)
     nc = nf # número de componetes (features) a utilizar
     col = list(range(nc))

     random.seed(w)  # 150
     rng=np.random.RandomState(j) # 155, 555

     acc_acu=0
     accuracy_acc=0
     N=int(len(data)-k)
     step=1
     i=0
     for n in range(0,N,step):
       i+=1
       wd=data
       #print('i',i)
       #print('shuffle in',data)
       ar=suffle_df(wd, 0, n, k)
       #print('shuffle out',data)
       X_train, X_test, y_train, y_test = train_test_sets(ar, 0, k, Eps, prop)
       #X_train, X_test, y_train, y_test = train_test_sets(ar, 0, k, prop)
       #print('X_train=', X_train.shape[0])
       # --------------------------------------------------------------
       ##model = DecisionTreeClassifier(max_depth = 15)
       ##model = RandomForestClassifier()
       ##model = LinearSVC(C=2, tol = 1e-5, max_iter = 500000)
       ##model = SVC(kernel='poly') # random.seed(50), pr=0.2, rand in augmented data=117 : 0.887
      #  model = GaussianNB() # norm01: 0.873239, 0.859, 0.8450, 0.8169

       ##model = MultinomialNB(class_prior=[0.55, 0.45])  # class_prior can be used to deal with class inbalance
       ##model = LogisticRegression(C=5, tol = 1e-5, max_iter = 1000000)
       ##model = LogisticRegressionCV(cv=CV, solver='liblinear', scoring='accuracy', random_state=random.randint(1,1000), n_jobs=-1, verbose=0, max_iter=LR_max_iter) # norm01: 0.85915 
       ##model = LogisticRegressionCV(cv=CV, scoring='accuracy', random_state=random.randint(1,1000), n_jobs=-1, verbose=0, max_iter=LR_max_iter) # norm01 87.3
       model = SVC(kernel='rbf') # random.seed(50), pr=0.2, rand in augmented data=117 : 0.887

       #Fitting de Random Forest
       model.fit(X_train, y_train)

       acc = model.score(X_test, y_test)
       acc_acu += acc

       predictions = model.predict(X_test)
       cm = confusion_matrix(y_test, predictions)
       accuracy= metrics.accuracy_score(y_test, predictions)
       accuracy_acc += accuracy
     #print('w=',w,'j=',j,"average accuracy=",acc_acu/i)

     if ((w+j)%100==0):
       print('w=',w,'j=',j,"average accuracy=",acc_acu/i)
       print('Execution time: ', time.time()-start_time)


     acc_ave=acc_acu/i
     if acc_ave > acc_max:
       acc_max=acc_ave
       print('****** w=',w,'j=',j,"average accuracy=",acc_acu/i,'************')


w= 200 j= 200 average accuracy= 0.6328125
Execution time:  8.469433307647705
****** w= 200 j= 200 average accuracy= 0.6328125 ************
****** w= 200 j= 201 average accuracy= 0.65625 ************
